In [1]:
import time
import warnings
from functools import partial

import matplotlib as mpl
import numpy as np
from IPython import get_ipython

import pymor.tools.random

ip = get_ipython()
if ip is not None:
#    ip.run_line_magic('matplotlib', 'inline')
    ip.run_line_magic('matplotlib', 'ipympl')

warnings.filterwarnings('ignore', category=UserWarning, module='torch')

pymor.tools.random._default_random_state = None

mpl.rcParams['figure.facecolor'] = (1.0, 1.0, 1.0, 0.0)
#mpl.rcParams['figure.dpi'] = 200

from pymor.core.logger import getLogger, set_log_levels

set_log_levels({ # disable logging for some components
    'main': 'DEBUG',
    'pymor': 'WARN',
    'pymor.models': 'WARN',
    'pymor.discretizers.builtin': 'WARN',
    'pymor.discretizers.dunegdt': 'DEBUG',
    'pymor.analyticalproblems.functions.BitmapFunction': 'ERROR',
    'models.ann.ANNStateReductor': 'INFO',
    'models.vkoga.VkogaStateModel': 'INFO',
    'models.vkoga.VkogaStateReductor': 'DEBUG',
    'models.adaptive': 'DEBUG',
    'algorithms.optimization': 'DEBUG'})

logger = getLogger('main.main')

In [2]:
# FOM
num_refines = 2
num_timesteps = 20

In [3]:
from pymor.basic import *


def setup_problem_and_discretize(num_refines, nt):
    from spe10channel import discretize, make_problem

    grid, num_grid_elements, boundary_info, problem, parameter_space, mu_bar = make_problem(
        regime='diffusion dominated', num_global_refines=num_refines)

    fom, fom_data, coercivity_estimator = discretize(
        grid, num_grid_elements, boundary_info, problem, mu_bar, nt=nt)

    return parameter_space, fom, fom_data, coercivity_estimator


spatial_product = lambda m: m.energy_0_product

In [4]:
logger.info('creating FOM:')
tic = time.perf_counter()

parameter_space, fom, fom_data, coercivity_estimator = setup_problem_and_discretize(
    num_refines, num_timesteps)

fom_offline_time = time.perf_counter() - tic
logger.info(f'  discretizing took {fom_offline_time}s')
logger.info(f'  grid has {fom_data["grid"].size(0)} elements,'
            f'FOM has {fom.solution_space.dim} DoFs, '
            f'uses {fom.time_stepper.nt} time steps')

logger.info(f'  input parameter space is {parameter_space.parameters.dim}-dimensional:')
logger.info(f'    {parameter_space}')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [5]:
logger.info('computing dual norm of output functional:')

assert not fom.output_functional.parametric
riesz_representative = spatial_product(fom).apply_inverse(fom.output_functional.as_vector())
dual_norm_output = np.sqrt(spatial_product(fom).apply2(riesz_representative, riesz_representative)[0][0])
del riesz_representative

logger.info(f'  {dual_norm_output}')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [6]:
#mu_ref = parameter_space.sample_uniformly(1)[0]
mu_init = fom.parameters.parse([2.0, 10.5])
mu_ref = fom.parameters.parse([5.005, 10])
#mu_opt = fom.parameters.parse([3.578, 10.095])
mu_opt = fom.parameters.parse([4.7421, 9.5263])
#mu_ref = fom.parameters.parse([10., 9.])

logger.info(f'computing f_h(mu={mu_ref}) ...')

tic = time.perf_counter()
f_mu_ref = fom.output(mu=mu_ref)#, incremental=True)  # no need to keep the state trajectory in memory
fom_online_output_time = time.perf_counter() - tic

logger.info(f'average FOM output (solve + apply functional) time: {fom_online_output_time}s')

logger.info('computing ||f_mu_ref||_{L^2(0, T)}:')
initial_abs_output_tol = fom.output_l2_norm(f_mu_ref)
logger.info(f'  {initial_abs_output_tol}')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [7]:
from pymor.models.hierarchy import AdaptiveModelHierarchy

In [8]:
rb_reductor = ParabolicRBReductor(fom, product=spatial_product(fom), coercivity_estimator=coercivity_estimator)

def reduction_rb(training_data, len_previous_training_data, models, reductors):
    U = fom.solution_space.empty(reserve=len(training_data))
    for _, u in training_data[len_previous_training_data:]:
        U.append(u)
    RB, _ = pod(U, product=fom.h1_0_semi_product)
    reductors[0].extend_basis(RB)
    # TODO: Use HaPOD here!!!
    return reductors[0].reduce()

def post_reduction_rb(training_data, models, reductors):
    return []

tolerance = 5e-3

# Settings for the two-stage hierarchy
models = [rb_reductor.reduce(), fom]
model_names = ['RB-ROM', 'FOM']
reductors = [rb_reductor]
reduction_methods = [reduction_rb]
post_reduction_methods = [post_reduction_rb]
training_frequencies = [1]

two_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                             training_frequencies, tolerance, visualizer=fom.visualizer,
                                             name='Two-stage model hierarchy')

In [9]:
def Linf_misfit(m, x, f_ref=None):
    mu = m.parameters.parse(x)
    f_x = m.output(mu)#, incremental=True)
    if f_ref is None:
        f_ref = m.output(mu=mu_ref)#, incremental=True)
    return np.max(np.abs(f_ref - f_x))

In [10]:
def objective_function(m, x):
    mu = m.parameters.parse(x)
    f_x = m.output(mu)
    return fom.T * np.mean(f_x) + 0.0075 * x["Da"][0] - 0.0375 * x["Pe"][0]

In [11]:
mean = (5.005, 10)
cov = [[2, 0], [0, 1]]

from scipy.stats import multivariate_normal
rv = multivariate_normal(mean, cov)

def random_sampling_function():
    return rv.rvs()

density_function_monte_carlo = rv.pdf

In [12]:
from pymor.models.interact import interact_model_hierarchy

In [13]:
"""
num_params = 20
uniform_params = parameter_space.sample_uniformly(num_params)
res = np.zeros((num_params, num_params))
for i, mu in enumerate(uniform_params):
    output = objective_function(fom, mu)
    res[(i-(i%num_params))//num_params, i%num_params] = output

import matplotlib.pyplot as plt
plt.close()
fig, axs = plt.subplots()
mp = axs.imshow(res, origin='lower')
fig.colorbar(mp, ax=axs)
plt.show()
"""

"\nnum_params = 20\nuniform_params = parameter_space.sample_uniformly(num_params)\nres = np.zeros((num_params, num_params))\nfor i, mu in enumerate(uniform_params):\n    output = objective_function(fom, mu)\n    res[(i-(i%num_params))//num_params, i%num_params] = output\n\nimport matplotlib.pyplot as plt\nplt.close()\nfig, axs = plt.subplots()\nmp = axs.imshow(res, origin='lower')\nfig.colorbar(mp, ax=axs)\nplt.show()\n"

In [14]:
"""
res_alt = res.copy()
params_array = np.array([p.to_numpy() for p in uniform_params]).reshape((num_params, num_params, 2))
#res_alt += 0.0075 * params_array[..., 0] - 0.0375 * params_array[..., 1]# - 9.)
#print(f"Matrix of output values: {res_alt}")
print(f"Minimum value: {np.min(res_alt)}")
amin = np.unravel_index(res_alt.argmin(), res_alt.shape)
print(f"Minimum index: {amin}")
print(f"Minimum parameter: {uniform_params[amin[0]*num_params+amin[1]]}")
plt.close()
fig_alt, axs_alt = plt.subplots(1, 2)
mp = axs_alt[0].imshow(res.T, origin='lower')
fig_alt.colorbar(mp, ax=axs_alt[0])
mp_alt = axs_alt[1].imshow(res_alt.T, origin='lower')
fig_alt.colorbar(mp_alt, ax=axs_alt[1])
plt.show()
"""

'\nres_alt = res.copy()\nparams_array = np.array([p.to_numpy() for p in uniform_params]).reshape((num_params, num_params, 2))\n#res_alt += 0.0075 * params_array[..., 0] - 0.0375 * params_array[..., 1]# - 9.)\n#print(f"Matrix of output values: {res_alt}")\nprint(f"Minimum value: {np.min(res_alt)}")\namin = np.unravel_index(res_alt.argmin(), res_alt.shape)\nprint(f"Minimum index: {amin}")\nprint(f"Minimum parameter: {uniform_params[amin[0]*num_params+amin[1]]}")\nplt.close()\nfig_alt, axs_alt = plt.subplots(1, 2)\nmp = axs_alt[0].imshow(res.T, origin=\'lower\')\nfig_alt.colorbar(mp, ax=axs_alt[0])\nmp_alt = axs_alt[1].imshow(res_alt.T, origin=\'lower\')\nfig_alt.colorbar(mp_alt, ax=axs_alt[1])\nplt.show()\n'

In [15]:
"""
fig = plt.figure(frameon=False)
fig.set_size_inches(1, 1)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(res.T, origin='lower', aspect='auto')
fig.savefig('fig_optimization_background_new.png', dpi=100)
"""

"\nfig = plt.figure(frameon=False)\nfig.set_size_inches(1, 1)\nax = plt.Axes(fig, [0., 0., 1., 1.])\nax.set_axis_off()\nfig.add_axes(ax)\nax.imshow(res.T, origin='lower', aspect='auto')\nfig.savefig('fig_optimization_background_new.png', dpi=100)\n"

# Katalytische Filter

Problem description!!!

# Adaptive Modellhierarchien

<style>.myst, .myst p {font-size: 20pt;}</style>

## Modelle

### Full order model (FOM)

### Reduced order model (RB-ROM)

### Machine learning model (ML-ROM)

Description + visualization

| Modell | Genauigkeit | Rechenaufwand | Sonstiges |
|--------|-------------|---------------|-----------|
| FOM    | Hinreichend genau für alle Parameter (Referenzlösung) | Sehr aufwendig | - |
| RB-ROM | Etwas ungenauer als das FOM | Deutlich schneller als das FOM | Effiziente Fehlerschätzung möglich |
| ML-ROM | Ungenauer als das RB-ROM | Schneller als das RB-ROM | Fehlerschätzung vom RB-ROM kann wiederverwendet werden |

In [16]:
"""
interact_model_hierarchy(two_stage_hierarchy, parameter_space, model_names,
                         output_function=lambda o: np.max(o), objective_function=partial(Linf_misfit, f_ref=f_mu_ref), initial_parameter=mu_init, optimal_parameter=mu_ref, optimization_bg_image="fig_optimization_adaptive_eps_bg.png", optimization_bg_image_limits=(0.0278514264616242, 0.000111348743739251), show_solution=True, visualizer=fom.visualizer, random_sampling_function=random_sampling_function, density_function_monte_carlo=density_function_monte_carlo, solution_plot_extent=(0, 5, 0, 1), language='de')
"""
#interact_model_hierarchy(two_stage_hierarchy, parameter_space, model_names,
#                         output_function=lambda o: np.max(o), objective_function=objective_function, initial_parameter=mu_init, optimal_parameter=mu_opt, optimization_bg_image="fig_optimization_background_new.png", optimization_bg_image_limits=(0.0278514264616242, 0.000111348743739251), show_solution=True, visualizer=fom.visualizer, random_sampling_function=random_sampling_function, density_function_monte_carlo=density_function_monte_carlo, solution_plot_extent=(0, 5, 0, 1), language='de')

'\ninteract_model_hierarchy(two_stage_hierarchy, parameter_space, model_names,\n                         output_function=lambda o: np.max(o), objective_function=partial(Linf_misfit, f_ref=f_mu_ref), initial_parameter=mu_init, optimal_parameter=mu_ref, optimization_bg_image="fig_optimization_adaptive_eps_bg.png", optimization_bg_image_limits=(0.0278514264616242, 0.000111348743739251), show_solution=True, visualizer=fom.visualizer, random_sampling_function=random_sampling_function, density_function_monte_carlo=density_function_monte_carlo, solution_plot_extent=(0, 5, 0, 1), language=\'de\')\n'

In [17]:
from pymor.reductors.ml import MachineLearningInstationaryReductor

method = 'gpr'
model_parameters = {}

ml_reductor = MachineLearningInstationaryReductor(fom=fom, training_set=None, validation_set=None, method=method, model_parameters=model_parameters, product=fom.h1_0_semi_product)

def reduction_ml(training_data, len_previous_training_data, models, reductors):
    rb_rom = models[1]
    rb_reductor = reductors[1]
    ml_reductor = reductors[0]
    error_estimator = rb_rom.error_estimator
    ml_reductor.reduced_basis = rb_reductor.bases['RB']
    red_dim = len(ml_reductor.reduced_basis)
    training_data = [(mu, np.pad(dat.to_numpy()[0], (0, red_dim - len(dat.to_numpy()[0])),
                                 mode='constant', constant_values=0.))
                     for (mu, dat) in training_data if np.linalg.norm(dat.to_numpy().flatten()) > 1e-10]
    ml_reductor.training_data = training_data
    ml_rom = ml_reductor.reduce()
    return ml_rom.with_(error_estimator=error_estimator, T=rb_rom.T, time_stepper=rb_rom.time_stepper)

def post_reduction_ml(training_data, models, reductors):
    return []

def post_reduction_rb(training_data, models, reductors):
    return [models[0]]
    #return [reduction_ml(training_data[1], None, models, reductors)]

models = [None, rb_reductor.reduce(), fom]
model_names = ['ML-ROM', 'RB-ROM', 'FOM']
reductors = [ml_reductor, rb_reductor]
reduction_methods = [reduction_ml, reduction_rb]
post_reduction_methods = [post_reduction_ml, post_reduction_rb]
training_frequencies = [10, 1]

three_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                               training_frequencies, tolerance)

In [18]:
interact_model_hierarchy(three_stage_hierarchy, parameter_space, model_names,
                         output_function=lambda o: np.max(o), objective_function=partial(Linf_misfit, f_ref=f_mu_ref), initial_parameter=mu_init, optimal_parameter=mu_ref, optimization_bg_image="fig_optimization_adaptive_eps_bg.png", optimization_bg_image_limits=(0.0278514264616242, 0.000111348743739251), show_solution=True, visualizer=fom.visualizer, random_sampling_function=random_sampling_function, density_function_monte_carlo=density_function_monte_carlo, solution_plot_extent=(0, 5, 0, 1), language='en',
                         fig_width=17)

In [ ]:
"""
from pymor.reductors.neural_network import NeuralNetworkInstationaryReductor
ml_reductor = NeuralNetworkInstationaryReductor(fom=fom, training_set=None, validation_set=None,
                                                ann_mse=None, pod_params={'product': fom.h1_0_semi_product},
                                                scale_inputs=False, scale_outputs=False)

def reduction_ml(training_data, len_previous_training_data, models, reductors):
    rb_rom = models[1]
    rb_reductor = reductors[1]
    ml_reductor = reductors[0]
    error_estimator = rb_rom.error_estimator
    ml_reductor.reduced_basis = rb_reductor.bases['RB']
    red_dim = len(ml_reductor.reduced_basis)
    training_data = [(mu, np.pad(dat.to_numpy()[0], (0, red_dim - len(dat.to_numpy()[0])),
                                 mode='constant', constant_values=0.))
                     for (mu, dat) in training_data]
    ml_reductor.training_data = training_data
    ml_rom = ml_reductor.reduce(restarts=2, log_loss_frequency=10, recompute_training_data=False,
                                recompute_validation_data=True)
    return ml_rom.with_(error_estimator=error_estimator, T=rb_rom.T, time_stepper=rb_rom.time_stepper)

def post_reduction_ml(training_data, models, reductors):
    return []

def post_reduction_rb(training_data, models, reductors):
    return [models[0]]
    #return [reduction_ml(training_data[1], None, models, reductors)]

models = [None, rb_reductor.reduce(), fom]
model_names = ['ML-ROM', 'RB-ROM', 'FOM']
reductors = [ml_reductor, rb_reductor]
reduction_methods = [reduction_ml, reduction_rb]
post_reduction_methods = [post_reduction_ml, post_reduction_rb]
training_frequencies = [20, 1]

three_stage_hierarchy = AdaptiveModelHierarchy(models, reductors, reduction_methods, post_reduction_methods,
                                               training_frequencies, tolerance)
"""

In [ ]:
"""
interact_model_hierarchy(three_stage_hierarchy, parameter_space, model_names,
                         output_function=lambda o: np.max(o), objective_function=partial(Linf_misfit, f_ref=f_mu_ref), initial_parameter=mu_init, optimal_parameter=mu_ref, optimization_bg_image="fig_optimization_adaptive_eps_bg.png", optimization_bg_image_limits=(0.0278514264616242, 0.000111348743739251), show_solution=True, visualizer=fom.visualizer, random_sampling_function=random_sampling_function, density_function_monte_carlo=density_function_monte_carlo, solution_plot_extent=(0, 5, 0, 1), language='de')
"""